In [1]:
from fredapi import Fred
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import os

In [2]:
fred = Fred(api_key=os.environ['FRED_API_KEY'])

In [6]:

# Specify the FRED series IDs and their corresponding maturity periods
series_ids = ["FEDFUNDS", "DGS1", "DGS3", "DGS5", "DGS7", "DGS10", "DGS30"]
maturities = [0, 1, 3, 5, 7, 10, 30]

# Fetch the series data from FRED
data = []
for series_id, maturity in zip(series_ids, maturities):
    series = fred.get_series(series_id)
    series.name = maturity
    data.append(series)
df = pd.concat(data, axis=1)

# Fetch the CPI series and calculate the 12 period percent change
cpi = fred.get_series("CPIAUCSL")
cpi_pct_change = cpi.pct_change(periods=12)  # 12 periods percent change
cpi_pct_change = cpi_pct_change * 100  # Convert to percentage
cpi_pct_change.name = 'inflation_rate'

# Join the CPI percent change series to the main DataFrame
df = df.join(cpi_pct_change)

# Drop NaNs from the DataFrame
df = df.dropna()

# Convert date index to string
df.index = df.index.date

# Reshape DataFrame for plotting
df_melt = df.reset_index().melt(id_vars=['index', 'inflation_rate'], var_name='maturity', value_name='yield')

# Pivot the DataFrame to create a uniform grid of x (maturity), y (date) and z (yield)
pivot_df = df_melt.pivot_table(values='yield', index='index', columns='maturity')

# Extract x, y and z values for the plot
x_values = pivot_df.columns.tolist()
y_values = pivot_df.index.tolist()
z_values = pivot_df.values



In [9]:
# Convert z values to percentages
z_values_percent = z_values * 1

# Create 3D surface plot
fig = go.Figure(data=[
    go.Surface(
        x=x_values,
        y=y_values,
        z=z_values_percent,
        surfacecolor=df_melt.pivot_table(values='inflation_rate', index='index', columns='maturity').values,
        colorscale='Jet',  # Feel free to change the color scale
        colorbar=dict(title='Inflation Rate (%)'), # Add a label to the colorbar
    )
])

fig.update_layout(title='3D - Yield Curve with Inflation Rate @MMTmacrotrader', autosize=True,
                  scene=dict(xaxis_title='Maturity',
                             yaxis_title='Date',
                             zaxis_title='Yield (%)'), # Update the z-axis label
                  width=1600,  # Set the width of the plot
                  height=900,  # Set the height of the plot
                  template='plotly_dark')  # Set the theme to dark mode

fig.show()

# Export the plot to HTML
fig.write_html("plot.html")